In [1]:
# Import Needed Modules
import pandas as pd
from dotenv import load_dotenv
import os, time, requests, csv
from datetime import datetime
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from functions import *

# Load Environments, API Key, and variables
load_dotenv('D:/AI/stock_market/.env')
API_KEY = os.getenv("ALPHA_PREM_API_KEY")

# Assign listing_info directory
listing_info_loc = "resources/listing_info.xlsx"

# Check if listing_info.xlsx exists.  If so pull the information, it not, create.
if os.path.exists(listing_info_loc) is False:
    # Run function to pull all listings and return dataframe
    df = get_market_listings(API_KEY)
else:
    # Import file of stock listings
    df = get_listings()

# Get List of technology stocks
tech = pd.read_excel("resources/technology_industry_list.xlsx")
tech_list = tech['symbol'].tolist()

In [4]:
# Feed list of symbols into function
companies_df = get_company_overview(tech_list, API_KEY)

In [5]:
companies_df['Beta'].replace('None', 0, inplace=True)
#test = test.fillna(0)
companies_df = companies_df.astype({"200DayMovingAverage":float, "Beta":float})
vol_companies_df = companies_df.loc[companies_df['200DayMovingAverage'] >= 150][['Symbol','Beta','200DayMovingAverage']].reset_index(drop=True)
vol_companies_df = vol_companies_df.sort_values("Beta", ascending=False)
vol_companies_df.head(20)

,Symbol,Beta,200DayMovingAverage
43,MSTR,2.617,445.83
46,ZBRA,1.776,255.60
67,ACLS,1.688,150.58
2,NVDA,1.684,484.29
33,HUBS,1.607,522.03
55,WEX,1.597,189.58
22,NXPI,1.578,205.33
64,NSIT,1.549,156.86
59,OLED,1.512,160.52
12,LRCX,1.504,694.82


In [6]:
for index, row in vol_companies_df[:20].iterrows():
    symbol = row['Symbol']

    # Create folder in charts to house each stock chart
    folder = f'charts/{symbol}'
    os.makedirs(folder, exist_ok=True)
    
    # Pull stock data
    # Since we are taking the top 20, it remains under 30 per min API request
    df_stock, df_daily = get_daily_series([symbol], API_KEY)

    # Run forecast for stocks on next trading day
    forecast, forecast_range = prediction_series(df_stock, 'volatility_diff', 288)

    # Get chart for components
    components = forecast_range.reset_index(drop=True).plot(x='ds', 
                                                            xlabel='Time of Day', 
                                                            ylabel="Volatility", 
                                                            figsize=[10,6],
                                                            title=f"Forecast Volatility Times ({symbol})")
    
    # Save chart and close figure
    components.figure.savefig(f'{folder}/{symbol}_forecast_upper_lower.jpg')
    plt.close(components.figure)

    # Get chart for yhat only
    yhat_component = forecast_range[['ds','yhat']].reset_index(drop=True).plot(x='ds', 
                                                            xlabel='Time of Day', 
                                                            ylabel="Volatility",
                                                            figsize=[10,6], 
                                                            title=f"yhat Only Forecast Volatility Times ({symbol})")

    # Save chart and close figure
    yhat_component.figure.savefig(f'{folder}/{symbol}_forecast_yhat.jpg')
    plt.close(yhat_component.figure)

14:42:21 - cmdstanpy - INFO - Chain [1] start processing
14:42:22 - cmdstanpy - INFO - Chain [1] done processing
14:42:26 - cmdstanpy - INFO - Chain [1] start processing
14:42:27 - cmdstanpy - INFO - Chain [1] done processing
14:42:31 - cmdstanpy - INFO - Chain [1] start processing
14:42:32 - cmdstanpy - INFO - Chain [1] done processing
14:42:37 - cmdstanpy - INFO - Chain [1] start processing
14:42:37 - cmdstanpy - INFO - Chain [1] done processing
14:42:42 - cmdstanpy - INFO - Chain [1] start processing
14:42:42 - cmdstanpy - INFO - Chain [1] done processing
14:42:46 - cmdstanpy - INFO - Chain [1] start processing
14:42:47 - cmdstanpy - INFO - Chain [1] done processing
14:42:52 - cmdstanpy - INFO - Chain [1] start processing
14:42:52 - cmdstanpy - INFO - Chain [1] done processing
14:42:56 - cmdstanpy - INFO - Chain [1] start processing
14:42:57 - cmdstanpy - INFO - Chain [1] done processing
14:43:03 - cmdstanpy - INFO - Chain [1] start processing
14:43:04 - cmdstanpy - INFO - Chain [1]